In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import firm_revised

In [90]:
seed = 100

In [91]:
df = pd.read_csv(f'/Users/popovici/Desktop/Thesis/markdown_estimation/data/market_{seed}.csv')
print(df)

         price      cost  product  time      Car1      Car2    mshare  \
0     9.552859  1.845445        1     1  5.543405  2.810504  0.005380   
1     3.064592  1.761890        2     1  5.278369  3.255001  0.090384   
2     2.000000  2.000000        3     1  5.424518  2.541973  0.206187   
3     1.940064  1.940064        4     1  5.844776  3.435163  0.067304   
4    10.000000  1.616043        5     1  5.004719  2.416405  0.011396   
..         ...       ...      ...   ...       ...       ...       ...   
985   1.165361  1.165361        6    99  5.121569  3.816847  0.005733   
986   1.356730  1.356730        7    99  5.670749  3.672721  0.004404   
987   1.004820  1.004820        8    99  5.825853  2.895589  0.010860   
988   1.358923  1.358923        9    99  5.136707  2.468720  0.006016   
989   1.098656  1.098656       10    99  5.575093  4.029733  0.004738   

      profits   markups  
0    0.041466  4.176452  
1    0.117743  0.739378  
2    0.000000  0.000000  
3    0.000000  0.00

In [92]:
# All the parameters of interest 
beta = np.array([2, -0.5, -0.3])
mu = 0.5
omega = 1


# The number of firms in the market
J = 10

# The number of product characteristics 
K = 2

# The number of consumers in the market 
N = 500

# The number of period that I want to run the market for 
T = 100

In [93]:
# Unpacking the X and the price
X1 = np.array(df['Car1'][:J])
X2 = np.array(df['Car2'][:J])
all_X = np.column_stack((X1, X2))
X0 = np.ones(J)
X = np.column_stack((X0, all_X))

eq_price = np.array(df['price'])
print(eq_price.shape)

true_share = np.array(df['mshare'])
print(true_share.shape)

all_est_shares = np.zeros(J*(T-1))
print(all_est_shares.shape)


(990,)
(990,)
(990,)


In [94]:
# The idea is that you want to min the distance between the true share and the made up share by optimizing the fiven vector theta that determines the share 

def f(theta, true_share, N, J, X, price):
    beta = theta[:K+1]
    mu = theta[K+1]
    omega = theta[K+2]
    # make a numpy array all estimated shares 
    for t in range(0, T-1):
        eq_price_slice = price[t*J:(t+1)*J]
        v_p = np.random.normal(0, 1, size = N)
        e = np.random.gumbel(size=N*J)
        est_share = firm_revised.share(N, J, X, v_p, eq_price_slice, beta, mu, omega, e)
        all_est_shares[t*J:(t+1)*J] = est_share

    return np.linalg.norm(true_share - all_est_shares)

In [95]:
theta_0 = np.array([1., 1., 1., 1., 1.])

In [118]:
all_val_b1 = []
for l in range(1, 100):
    res = minimize(f, theta_0, args=(true_share, N, J, X, eq_price), method = 'Nelder-Mead')
    optim_vec = res.x
    all_val_b1.append(optim_vec[1])

arr = np.array(all_val_b1)
print(np.mean(arr))


KeyboardInterrupt: 

In [119]:
print(all_val_b1)

[1.0, 1.0230676, 0.9999958977056864, 0.9922742318208615, 1.0012949711870598, 1.0080271756617294, 1.013434939394127, 0.981244338479099, 1.0017964417776528, 1.0000001592633263, 0.99902179725306, 0.993752897280106, 1.0104584073747183, 1.0016597030699848, 1.0498217015680105, 1.0150122000000001, 0.999994682609852, 1.011809701635352, 1.0377821544022272, 1.0088364403859975, 1.0076472452294258, 1.0283371241771153, 1.0242016072062068, 1.0467349022815193, 1.0044300210177988, 1.052653885402584, 1.0150000325988975, 0.9986027544752811, 1.0072969377091725, 1.0040662323760317, 1.0017577032274418, 1.0423992680491492, 1.0090659256312837, 1.0308560048557727, 0.9835559575710032, 1.049297334719113, 1.0198508257483772, 1.025811714249568, 1.0086309007701302, 1.00001140222175, 0.9940779850093081, 0.9908772079873038, 1.0109946486991799, 1.006472, 0.9999210364382835, 0.9999969064044381, 1.0044875351266156, 0.9998085886515029, 0.9093495153919995, 1.0093064005145505, 0.9975483700396552, 1.0641812613307853, 1.011